In [10]:
import mercantile, mapbox_vector_tile, requests, json
from vt2geojson.tools import vt_bytes_to_geojson
import os
output_folder_path = os.path.join('..', '..','data','02_intermediate_output') 

In [ ]:
# enter your acccess token
access_token = 'MLY|xxxxxxxxxxxxxxxxxxx'

# define an empty geojson as output
output= { "type": "FeatureCollection", "features": [] }

In [4]:
# a bounding box in [east_lng,_south_lat,west_lng,north_lat] format
# our bbox is whole of RDJ
west, south, east, north = [-43.77359702712855,-23.21292529817678,-43.030218255454905,-22.676377767232317]
tiles = list(mercantile.tiles(west, south, east, north, 14)) # zoom level 14

# vector tile endpoints 
tile_points = 'mly_map_feature_point'
tile_layer = "point"

# values to filter from API
filter_values = ['object--street-light', 'object--trash-can', 'object--cctv-camera']

In [5]:
# loop through list of tiles 
for tile in tiles:
    tile_url = 'https://tiles.mapillary.com/maps/vtp/{}/2/{}/{}/{}?access_token={}'.format(tile_points,tile.z,tile.x,tile.y,access_token)
    response = requests.get(tile_url)
    data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z,layer=tile_layer)

    # filter for only features matching the values in filter list above
    filtered_data = [feature for feature in data['features'] if feature['properties']['value'] in filter_values]

    # check if features are inside bbox, and push to output geojson object if yes
    for feature in filtered_data:
      if feature['geometry']['type'] == 'Point':
        output['features'].append(feature)



In [11]:
# save as geojson
output_filepath = os.path.join(output_folder_path, 'mapillary_points.geojson')
with open(output_filepath, 'w') as f:
    json.dump(output, f)